In [1]:
from stockfish import Stockfish

In [15]:
# settings
settings = {
    "Debug Log File": "",
    "Contempt": 0,
    "Min Split Depth": 0,
    "Threads": 4, # More threads will make the engine stronger, but should be kept at less than the number of logical processors on your computer.
    "Ponder": "false",
    "Hash": 2048, # Default size is 16 MB. It's recommended that you increase this value, but keep it as some power of 2. E.g., if you're fine using 2 GB of RAM, set Hash to 2048 (11th power of 2).
    "MultiPV": 1,
    "Skill Level": 20,
    "Move Overhead": 10,
    "Minimum Thinking Time": 20,
    "Slow Mover": 100,
    "UCI_Chess960": "false",
    "UCI_LimitStrength": "false",
    "UCI_Elo": 3400
}

In [16]:
stockfish = Stockfish(path="stockfish/stockfish-windows-x86-64-avx2.exe", depth=25, parameters=settings)
stockfish

In [17]:
stockfish.set_fen_position("rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2")

In [18]:
stockfish.get_best_move()

'd2d4'

In [19]:
stockfish.get_top_moves(3)

[{'Move': 'd2d4', 'Centipawn': 43, 'Mate': None},
 {'Move': 'b1c3', 'Centipawn': 36, 'Mate': None},
 {'Move': 'g1f3', 'Centipawn': 16, 'Mate': None}]

In [21]:
print(stockfish.get_board_visual())

+---+---+---+---+---+---+---+---+
| r | n | b | q | k | b | n | r | 8
+---+---+---+---+---+---+---+---+
| p | p | p | p |   | p | p | p | 7
+---+---+---+---+---+---+---+---+
|   |   |   |   | p |   |   |   | 6
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 5
+---+---+---+---+---+---+---+---+
|   |   |   |   | P |   |   |   | 4
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 3
+---+---+---+---+---+---+---+---+
| P | P | P | P |   | P | P | P | 2
+---+---+---+---+---+---+---+---+
| R | N | B | Q | K | B | N | R | 1
+---+---+---+---+---+---+---+---+
  a   b   c   d   e   f   g   h



In [24]:
stockfish.make_moves_from_current_position(["d2d4"])

In [25]:
print(stockfish.get_board_visual())

+---+---+---+---+---+---+---+---+
| r | n | b | q | k | b | n | r | 8
+---+---+---+---+---+---+---+---+
| p | p | p | p |   | p | p | p | 7
+---+---+---+---+---+---+---+---+
|   |   |   |   | p |   |   |   | 6
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 5
+---+---+---+---+---+---+---+---+
|   |   |   | P | P |   |   |   | 4
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 3
+---+---+---+---+---+---+---+---+
| P | P | P |   |   | P | P | P | 2
+---+---+---+---+---+---+---+---+
| R | N | B | Q | K | B | N | R | 1
+---+---+---+---+---+---+---+---+
  a   b   c   d   e   f   g   h



In [26]:
stockfish.get_top_moves(3)

[{'Move': 'd7d5', 'Centipawn': 42, 'Mate': None},
 {'Move': 'a7a6', 'Centipawn': 65, 'Mate': None},
 {'Move': 'c7c5', 'Centipawn': 70, 'Mate': None}]